### Entropy

Entropy is a measure of how `disordered` a collection is.  
The `more` impure the feature is, the higher the entropy.  

Probability distribution is the `frequency` of the unique values.  
It turns out that a `logarithm` of the number of states is perfect for disorder.  

$ H(X) = -\sum_{i=1}^{n} P(x_i) \log_2 P(x_i) $

In [83]:
""" Decision Tree / Entropy
"""

import pandas as pd
import numpy as np

# Set the initial traning data
A = ['apple']*1 + ['orange']*2 + ['banana']*2
B = ['apple']*5 + ['orange']*2 + ['banana']*0

# Probability (by hand)
P1 = [1/5, 2/5, 2/5] 
P2 = [5/7, 2/7, 0/7]

# Probability (with pandas)
A = pd.Series(A)
B = pd.Series(B)

P3 = A.value_counts(normalize=True)
P4 = B.value_counts(normalize=True)

# Entropy (Shannon model)
H3 = -1 * np.sum(P3 * np.log2(P3))
H4 = -1 * np.sum(P4 * np.log2(P4))
assert H3 > H4

# Output results
print("Datasets:")
print("A =", A.values)
print("B =", B.values)

print("\n Probability distributions (by hand):")
print(P1)
print(P2)

print("\n Probability distributions (with pandas):")
print(P3.values)
print(P4.values)

print("\n Entropies:")
print(H3)
print(H4)

Datasets:
A = ['apple' 'orange' 'orange' 'banana' 'banana']
B = ['apple' 'apple' 'apple' 'apple' 'apple' 'orange' 'orange']

 Probability distributions (by hand):
[0.2, 0.4, 0.4]
[0.7142857142857143, 0.2857142857142857, 0.0]

 Probability distributions (with pandas):
[0.4 0.4 0.2]
[0.71428571 0.28571429]

 Entropies:
1.5219280948873621
0.863120568566631


### Gini index

Both entropy and Gini index can be used as impurity `measures` for decision tree algorithms.  
Gini index is `between` 0 and 1, it easy to compare gini across different features.  

Gini index is often preferred due to its computational `efficiency`.  
Entropy may be more `sensitive` to changes in class probabilities.  

$ \text{Gini}(X) = 1 - \sum_{x \in X} P(x)^2 $

In [84]:
""" Decision Tree / Gini Index
"""

import pandas as pd
import numpy as np

# Set the initial traning data
A = ['apple']*1 + ['orange']*2 + ['banana']*2
B = ['apple']*5 + ['orange']*2 + ['banana']*0

A = pd.Series(A)
B = pd.Series(B)

# Probability distribution
P1 = A.value_counts(normalize=True)
P2 = B.value_counts(normalize=True)

# Gini Index
g1 = 1 - np.sum(np.square(P1)) # Look Here
g2 = 1 - np.sum(np.square(P2))
assert g1 > g2

# Output results
print("Datasets:")
print("A =", A.values)
print("B =", B.values)

print("\n Probability distributions (by hand):")
print(P1)
print(P2)

print("\n Gini indexes:")
print(g1)
print(g2)

Datasets:
A = ['apple' 'orange' 'orange' 'banana' 'banana']
B = ['apple' 'apple' 'apple' 'apple' 'apple' 'orange' 'orange']

 Probability distributions (by hand):
orange    0.4
banana    0.4
apple     0.2
dtype: float64
apple     0.714286
orange    0.285714
dtype: float64

 Gini indexes:
0.6399999999999999
0.40816326530612246


### Information gain

Information gain is a measure of the `reduction` in entropy.  
As the entropy of an attribute `increases`, the information gain decreases.  
We can find the most useful attribute and `split` the dataset based on that attribute.  

$ 
    \text{IG}(X, A) = \text{H}(X) - \sum_{v \in \text{Values}(A)} \frac{|X_v|}{|X|}  \cdot \text{H}(X_v) 
$

In [85]:
""" Decision Trees / Info Gain
Play Tennis

Play Tennis example (information gain for wind):
IG = H - (8/14)H_weak - (6/14)H_strong
IG = 0.940 - (8/14)0.811 - (6/14)1.00 = 0.048 

Machine epsilon is the upper bound on the relative error due to rounding.
This small value added to the denominator in order to avoid division by zero. 
"""

import numpy as np
import pandas as pd
import pathlib

# Load the dataset from a CSV file
df = pd.read_csv('decision_tree/data/play_tennis.csv')

# Split the dataset into features (X) and the target (y), whether play tennis or not
X = df.drop(['play'], axis=1)
y = df['play'] 

# Function to calculate the total entropy of the dataset
def dataset_entropy():
    E = 0

    # Count the occurrences of each play outcome (yes=9, no=5)
    N = df['play'].value_counts()

    # For each unique play outcome (yes/no)
    for v in df['play'].unique():

        # Calculate the probability of this outcome
        P = N[v]/len(df['play'])

         # Update total entropy using the probability
        E += -P*np.log2(P)
    return E

# Function to calculate the entropy of a specific attribute
def attribute_entropy(attr):
    E = 0

    # Calculate machine epsilon for float operations
    eps = np.finfo(float).eps 

    # Get unique play outcomes (yes/no)
    targets = df.play.unique()

    # Get unique values for the given attribute
    values = df[attr].unique()

    # For each unique value of the attribute (cool/hot)
    for v in values:

        # Initialize entropy for this value
        ent = 0

        # For each unique play outcome (yes/no)
        for t in targets:

            # Count occurrences where attribute=value and play outcome matches
            num = len(df[attr][df[attr] == v][df.play == t]) # numerator

            # Count occurrences where attribute=value
            den = len(df[attr][df[attr] == v])

            # Calculate a fraction related to the probability
            fraction = num/(den + eps)

            # Update entropy for this value
            ent += -fraction*np.log2(fraction + eps)

        # Update total entropy using the weighted entropy for this value
        E += -(den/len(df))*ent # sum of all entropies

    # Return the absolute value of the entropy
    return abs(E)


# Get the names of attributes (excluding the target variable)
attributes = df.keys()[:-1]

# Calculate entropy for each attribute and store it in a dictionary
E = {} 
for k in attributes:
    E[k] = attribute_entropy(k)

# Calculate information gain for each attribute and store it in a dictionary
IG = {}
for k in E:
    IG[k] = dataset_entropy() - E[k]

# Alternatives one-liner versions to calculate entropy and information gain
E  = {k:attribute_entropy(k) for k in attributes}
IG = {k:(dataset_entropy() - E[k]) for k in E} 

# Asserts
assert E['outlook']  < E['humidity']
assert IG['outlook'] > IG['humidity'] # Look Here

# Output results
print("\n Dataset:"); print(df)
print("\n Describe:"); print(df.describe())
print("\n Entropy:"); print(dataset_entropy())
print("\n AttrEntropy:"); print(E)
print("\n Information gains:"); print(IG)


 Dataset:
     outlook  temp humidity  windy play
0      sunny   hot     high  False   no
1      sunny   hot     high   True   no
2   overcast   hot     high  False  yes
3      rainy  mild     high  False  yes
4      rainy  cool   normal  False  yes
5      rainy  cool   normal   True   no
6   overcast  cool   normal   True  yes
7      sunny  mild     high  False   no
8      sunny  cool   normal  False  yes
9      rainy  mild   normal  False  yes
10     sunny  mild   normal   True  yes
11  overcast  mild     high   True  yes
12  overcast   hot   normal  False  yes
13     rainy  mild     high   True   no

 Describe:
       outlook  temp humidity  windy play
count       14    14       14     14   14
unique       3     3        2      2    2
top      sunny  mild     high  False  yes
freq         5     6        7      8    9

 Entropy:
0.9402859586706311

 AttrEntropy:
{'outlook': 0.6935361388961914, 'temp': 0.9110633930116756, 'humidity': 0.7884504573082889, 'windy': 0.892158928262361}

 